In [1]:
import requests
import lxml.html as lh
import pandas as pd
import csv

In [91]:
url = 'https://simplon.co/je-candidate'
#url sur lequel travaillé

uri_pages = '?page='
nbPages = 1
tags = ['//h2[@class="card-title"]', \
        '//div[@class="card-content"]', \
        '//span[@class="day"]', \
        '//span[@class="month"]', \
        '//div[@class="year"]', \
        '//div[@class="card-session-info calendar"]', \
        '//div[@class="card-session-infos"]']

cols = ['Nom', 'Lieu', 'Jour', 'Mois', 'Annee', 'Debut', 'Information']
 
page = requests.get(url)
doc = lh.fromstring(page.content)
 

In [73]:
test_xpath = []
test_xpath.append(doc.xpath('//div[@class="card-content"]'))
tmp = pd.DataFrame([test_xpath[0][i].text_content().strip()])

In [74]:
len(test_xpath[0])

73

In [75]:
#fonctionne 
content=[]
for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

In [76]:
len(content) #7 tags différents 

7

In [77]:
# d'un chargement à l'autre nombres de contents par tags  changeant
for i in range (len(tags)):
    print(i,len(doc.xpath(tags[i])))

0 73
1 73
2 73
3 73
4 73
5 73
6 73


### Récupère le contenu par tags et lui assigner une colonne 
un tag = une colonne 

In [86]:
df_liste

[                                                  Nom  key
 0   Concepteur développeur d'applications JavaScri...    0
 1   Concepteur développeur d'applications React / ...    1
 2   Technicien d’assistance en informatique - 1 mo...    2
 3   Technicien supérieur systèmes et réseaux - Alt...    3
 4   Administrateur·trice d’infrastructures sécuris...    4
 ..                                                ...  ...
 68   Méthodes agiles de gestion et amorçage de projet   68
 69  Méthodes agiles de gestion et amorçage de proj...   69
 70  Méthodes agiles de gestion et amorçage de proj...   70
 71   Méthodes agiles de gestion et amorçage de projet   71
 72  Méthodes agiles de gestion et amorçage de proj...   72
 
 [73 rows x 2 columns],
                                                  Lieu  key
 0   Alternance\n                            \n    ...    0
 1   Alternance\n                            \n    ...    1
 2   Alternance\n                            \n    ...    2
 3   Alternanc

In [78]:
# Récupère 1 tag = 1 colonne et remplit le contenu
df_liste = []
for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

###  Ajouter des lignes ensembles et les colonnes les unes après les autres

In [84]:
# Build the unique Dataframe with one tag (xpath) content per column
liste = df_liste[0]
for j in range(len(tags)-1):
    liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
    liste['key'] = liste.index
    del liste['key_l']
    del liste['key_r']

In [85]:
liste

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Concepteur développeur d'applications JavaScri...,Alternance\n \n ...,20,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,0
1,Concepteur développeur d'applications React / ...,Alternance\n \n ...,20,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,1
2,Technicien d’assistance en informatique - 1 mo...,Alternance\n \n ...,20,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,2
3,Technicien supérieur systèmes et réseaux - Alt...,Alternance\n \n ...,20,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,20,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,4
...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,event\n \n\n Déb...,event\n \n\n Déb...,68
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,event\n \n\n Déb...,event\n \n\n Déb...,69
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,event\n \n\n Déb...,event\n \n\n Déb...,70
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,event\n \n\n Déb...,event\n \n\n Déb...,71


# Fonction globale de récupération des données

In [92]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
 
    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)
 
    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste
 
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

In [93]:
liste_totale = getPages(nbPages, url)

In [94]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,0
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,1
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,2
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,3
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,4
...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,event\n \n\n Déb...,event\n \n\n Déb...,68
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,event\n \n\n Déb...,event\n \n\n Déb...,69
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,event\n \n\n Déb...,event\n \n\n Déb...,70
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,event\n \n\n Déb...,event\n \n\n Déb...,71


In [100]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'event':''}, regex= True)

In [101]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'\n':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({'Début':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({':':''}, regex= True)

In [102]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,event\n \n\n Déb...,0
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,event\n \n\n Déb...,1
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,event\n \n\n Déb...,2
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,event\n \n\n Déb...,3
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,event\n \n\n Déb...,4
...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,mars 2022,event\n \n\n Déb...,68
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,mai 2022,event\n \n\n Déb...,69
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,juin 2022,event\n \n\n Déb...,70
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,juin 2022,event\n \n\n Déb...,71


In [103]:
liste_totale['Information'] = liste_totale['Information'].replace({'event':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'\n':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'Début':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({':':''}, regex= True)

In [115]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,0
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,1
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,2
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,août 2021 ...,3
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,août 2021 ...,4
...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,mars 2022,mars 2022 ...,68
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,mai 2022,mai 2022 ...,69
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,juin 2022,juin 2022 ...,70
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,juin 2022,juin 2022 ...,71


### Création de colonnes, dérivée d'information

In [107]:
df = liste_totale.copy()
df["duree1"] = df["Information"].str.split().str[3]
df["duree2"] = df["Information"].str.split().str[4]
df["duree3"] = df["Information"].str.split().str[5]
df["duree4"] = df["Information"].str.split().str[6]

In [108]:
df

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key,duree1,duree2,duree3,duree4
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,0,au-delà,de,7,mois
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,1,au-delà,de,7,mois
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,2,au-delà,de,7,mois
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,août 2021 ...,3,inférieur,à,3,mois
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,août 2021 ...,4,inférieur,à,3,mois
...,...,...,...,...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,mars 2022,mars 2022 ...,68,inférieur,à,10,jours
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,mai 2022,mai 2022 ...,69,inférieur,à,10,jours
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,juin 2022,juin 2022 ...,70,inférieur,à,10,jours
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,juin 2022,juin 2022 ...,71,inférieur,à,10,jours


### Création de la colonne Localisation

In [109]:
df["Localisation"] = df["Information"].str.split().str[8]

In [110]:
df.head()

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key,duree1,duree2,duree3,duree4,Localisation
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,0,au-delà,de,7,mois,Auvergne-Rhône-Alpes
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,1,au-delà,de,7,mois,Auvergne-Rhône-Alpes
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,septembre 20...,2,au-delà,de,7,mois,Auvergne-Rhône-Alpes
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,août 2021 ...,3,inférieur,à,3,mois,Hauts-de-France
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,août 2021 ...,4,inférieur,à,3,mois,Auvergne-Rhône-Alpes


In [111]:
df = df.drop('Information', axis=1)

coller le contenu de chaque ligne entre elle dans la colonne duree

In [112]:
df['Duree'] = df['duree1'] + ' ' + df['duree2'] + ' ' + df['duree3'] + ' ' + df['duree4']

In [113]:
df.head()

,Nom,Lieu,Jour,Mois,Annee,Debut,key,duree1,duree2,duree3,duree4,Localisation,Duree
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,0,au-delà,de,7,mois,Auvergne-Rhône-Alpes,au-delà de 7 mois
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,1,au-delà,de,7,mois,Auvergne-Rhône-Alpes,au-delà de 7 mois
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,2,au-delà,de,7,mois,Auvergne-Rhône-Alpes,au-delà de 7 mois
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,3,inférieur,à,3,mois,Hauts-de-France,inférieur à 3 mois
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,4,inférieur,à,3,mois,Auvergne-Rhône-Alpes,inférieur à 3 mois


In [124]:
df = df.drop(['duree1', 'duree2', 'duree3', 'duree4',], axis=1)

In [125]:
df['datelimite'] = df['Jour'] + '-' + df['Mois'] + '-' + df['Annee']

In [126]:
df

,Nom,Lieu,Jour,Mois,Annee,Debut,key,Localisation,Duree,datelimite
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,septembre 2021,0,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,septembre 2021,1,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,septembre 2021,2,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,25,07,2021,août 2021,3,Hauts-de-France,inférieur à 3 mois,25-07-2021
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,août 2021,4,Auvergne-Rhône-Alpes,inférieur à 3 mois,27-07-2021
...,...,...,...,...,...,...,...,...,...,...
68,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,14,03,2022,mars 2022,68,Île-de-France,inférieur à 10 jours,14-03-2022
69,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,02,05,2022,mai 2022,69,Île-de-France,inférieur à 10 jours,02-05-2022
70,Méthodes agiles de gestion et amorçage de proj...,CPF\n \n ...,06,06,2022,juin 2022,70,Île-de-France,inférieur à 10 jours,06-06-2022
71,Méthodes agiles de gestion et amorçage de projet,CPF\n \n ...,06,06,2022,juin 2022,71,Île-de-France,inférieur à 10 jours,06-06-2022


In [127]:
df = df.drop(['Jour', 'Mois', 'Annee'], axis=1)

In [128]:
pd.set_option('display.max_rows', 56)

In [129]:
df = df.head(57)

In [130]:
df

,Nom,Lieu,Debut,key,Localisation,Duree,datelimite
0,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,septembre 2021,0,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
1,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,septembre 2021,1,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
2,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,septembre 2021,2,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-07-2021
3,Développeur·euse Front-end - Module de spécial...,Simplon Roubaix,août 2021,3,Hauts-de-France,inférieur à 3 mois,25-07-2021
4,Apple Foundation Program > Développement mobil...,Simplon Lyon,août 2021,4,Auvergne-Rhône-Alpes,inférieur à 3 mois,27-07-2021
...,...,...,...,...,...,...,...
52,Concepteur.trice développeur.euse d'applicatio...,Simplon Valenciennes - Les Ateliers Numérique,novembre 2021,52,Hauts-de-France,au-delà de 7 mois,28-10-2021
53,MS Technicien.ne IA PRF,Simplon Flagship Lille,novembre 2021,53,Hauts-de-France,inférieur à 3 mois,29-10-2021
54,Data visualisation ( Module BI ),CPF\n CNE\n...,novembre 2021,54,Grand,inférieur à 10 jours,01-11-2021
55,Développeur.euse COBOL PRF,Simplon Flagship Lille,novembre 2021,55,Hauts-de-France,inférieur à 3 mois,07-11-2021


In [ ]:
df.to_csv('formation_simplon.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [131]:
url = 'https://auvergnerhonealpes.simplon.co/candidatures.html'
#url sur lequel travaillé

uri_pages = '?page='
nbPages = 1
tags = ['//h2[@class="card-title"]', \
        '//div[@class="card-content"]', \
        '//span[@class="day"]', \
        '//span[@class="month"]', \
        '//div[@class="year"]', \
        '//div[@class="card-session-info calendar"]', \
        '//div[@class="card-session-infos"]']

cols = ['Nom', 'Lieu', 'Jour', 'Mois', 'Annee', 'Debut', 'Information']
 
page = requests.get(url)
doc = lh.fromstring(page.content)

In [139]:
test_xpath = []
test_xpath.append(doc.xpath('//div[@class="card-content"]'))

In [145]:
#9 formations
len(test_xpath[0])

9

In [146]:
tmp = pd.DataFrame([test_xpath[0][1].text_content().strip()])

In [147]:
tmp

,0
0,Simplon Lyon


In [156]:
i = 1
for elements in test_xpath[0] :
    i+=1
    print(elements.text_content().strip()) #.strip enleve les espaces entre les caracteres
    

Simplon V'E Le Cheylard
Simplon Lyon
AUVERGNE RHONE-ALPES
Simplon Lyon
Simplon Grenoble
Alternance
                            
            Simplon Lyon
Alternance
                            
            AUVERGNE RHONE-ALPES
Alternance
                            
            AUVERGNE RHONE-ALPES
Simplon Lyon


In [166]:
test_xpath[0][0].text_content().strip()

"Simplon V'E Le Cheylard"

In [ ]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
 
    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)
 
    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste
 
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

In [169]:
dataframe_reponse = getPages(1, 'https://auvergnerhonealpes.simplon.co/candidatures.html')

In [170]:
dataframe_reponse

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Développeur·se web et web mobile,Simplon V'E Le Cheylard,28,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,0
1,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,1
2,Parcours Welcode Auvergne Rhône-Alpes 2020 - c...,AUVERGNE RHONE-ALPES,31,08,2021,event\n \n\n Déb...,event\n \n\n Déb...,2
3,Apple Foundation Program > Développement mobil...,Simplon Lyon,05,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,3
4,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,06,09,2021,event\n \n\n Déb...,event\n \n\n Déb...,4
5,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,5
6,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,6
7,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,7
8,Hackeuses - Culture et techniques du numérique...,Simplon Lyon,20,09,2021,event\n \n\n Déb...,event\n \n\n Déb...,8


In [184]:
dataframe_reponse['Nom'].str.contains('100% DISTANCIEL')

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
8    False
Name: Nom, dtype: bool

In [192]:
dataframe_reponse.drop(['PRESENTIEL/DISTANCIEL'], axis =1,inplace=True)

In [233]:
dataframe_reponse['Nom'].loc[1]

'Apple Foundation Program > Développement mobile iOS >  Septembre 2021'

In [237]:
for index, i in enumerate (dataframe_reponse['Nom'].str.contains('100% DISTANCIEL')):
    print(i)
    if i == True:
        dataframe_reponse['PRESENTIEL/DISTANCIEL'].loc[index]= 'Distanciel'
    elif i == False :
        dataframe_reponse['PRESENTIEL/DISTANCIEL'].loc[index]= 'Presentiel'

False
False
False
False
False
False
True
True
False


C:\Users\utilisateur\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [238]:
dataframe_reponse

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key,PRESENTIEL/DISTANCIEL
0,Développeur·se web et web mobile,Simplon V'E Le Cheylard,28,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,0,Presentiel
1,Apple Foundation Program > Développement mobil...,Simplon Lyon,27,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,1,Presentiel
2,Parcours Welcode Auvergne Rhône-Alpes 2020 - c...,AUVERGNE RHONE-ALPES,31,08,2021,event\n \n\n Déb...,event\n \n\n Déb...,2,Presentiel
3,Apple Foundation Program > Développement mobil...,Simplon Lyon,05,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,3,Presentiel
4,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,06,09,2021,event\n \n\n Déb...,event\n \n\n Déb...,4,Presentiel
5,Concepteur développeur d'applications - Simplo...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,5,Presentiel
6,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,6,Distanciel
7,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,21,07,2021,event\n \n\n Déb...,event\n \n\n Déb...,7,Distanciel
8,Hackeuses - Culture et techniques du numérique...,Simplon Lyon,20,09,2021,event\n \n\n Déb...,event\n \n\n Déb...,8,Presentiel


### Afficher le lieu en question 

In [264]:
for j in  dataframe_reponse['Lieu']:
    print(j.split()[-1])

Cheylard
Lyon
RHONE-ALPES
Lyon
Grenoble
Lyon
RHONE-ALPES
RHONE-ALPES
Lyon
